In [ ]:
import tkinter as tk
from tkinter import ttk                      #Tkinter Themed Widgets
import sqlite3
import serial
import datetime
import matplotlib
import seaborn as sns
import quandl
import numpy as np
import scipy as sp
import pandas as pd
from pandas.tools.plotting import autocorrelation_plot
import sklearn.metrics
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline 
import warnings
        

def reg():
    conn = sqlite3.connect('pythonProject.db')
    facType = var.get()
    if(facType == '1'):
        name = 'Nuclear Plant'
    elif(facType == '2'):
        name = 'Textile Industry'
    elif(facType == '3'):
        name = 'Thermal Plant'
    elif(factType == '4'):
        name = 'Chemical Industry'
    if(conn.execute("INSERT INTO REGISTRATION (FIRSTNAME,LASTNAME,FACTORYNAME,FACTORYTYPE,TEMP,UNAME,PASSWORD) VALUES (?,?,?,?,?,?,?)",(fname.get(), lname.get(), cname.get(), name, temp.get(), uname.get(), password.get()))):
        tkLabel = tk.Label(tab1, text = "Registered successfully").grid(row = 10, column = 2)
        conn.commit()
        conn.close()
        fname.delete(0, 'end')
        lname.delete(0, 'end')
        cname.delete(0, 'end')
        temp.delete(0, 'end')
        uname.delete(0, 'end')
        password.delete(0, 'end')
        
def login():
    conn = sqlite3.connect('pythonProject.db')
    result = conn.execute('''SELECT * FROM REGISTRATION WHERE UNAME = ? AND PASSWORD = ?''',(unameLogin.get(), passwordLogin.get()))
    for row in result:
        if(row):
            tkLabel = tk.Label(tab2, text = "Welcome "+unameLogin.get()).grid(row = 10, column = 2)
            unameLogin.delete(0, 'end')
            passwordLogin.delete(0, 'end')
            tabControl.add(tab3, text = "Measure")
        else:
            tkLabel = tk.Label(tab2, text = "Invalid username or password").grid(row = 10, column = 2)
            
def logoutfun():
    tabControl.hide(tab3)
    
def predictFun():
    import quandl
    import numpy as np
    import pandas as pd
    from pandas.tools.plotting import autocorrelation_plot
    %matplotlib inline 
    import warnings
    warnings.filterwarnings("ignore")
    df=pd.read_csv("project.csv")
    df
    df.set_index("date",inplace=True)
    
    from statsmodels.tsa.arima_model import ARIMA
    model = ARIMA(df.value,order=(4,1,1))
    model_fit = model.fit(disp=0)
    arima_pred = model_fit.predict(df['value'].index[80], df['value'].index[-1], typ="levels")
    df_new = df[79:-1]
    tkLabel = tk.Label(tab3, text = "Time").grid(row = 4, column = 0)
    tkLabel = tk.Label(tab3, text = "Actual value").grid(row = 4, column = 1)
    tkLabel = tk.Label(tab3, text = "Predicted value").grid(row = 4, column = 2)
    for i in range(0,20):
        tkLabel = tk.Label(tab3, text = df_new.index[i]).grid(row = i+5, column = 0)
        tkLabel = tk.Label(tab3, text = df_new.iloc[i].value).grid(row = i+5, column = 1)
        tkLabel = tk.Label(tab3, text = arima_pred[i]).grid(row = i+5, column = 2)
    logout = tk.Button(tab3, text = "Logout", command = logoutfun).grid(row = i+7,column = 2)
    
def measure():
    ser=serial.Serial('COM8',9600)
    i=1
    file = open("project.csv", "w")
    file.write(("date,value\n"))
    while (i<=100):
        while(ser.inWaiting()>0):
            now = datetime.datetime.now()
            data=str(ser.readline())
            a=float(data[2:7])
            print(a)
            file.write(str(now))
            file.write(',')
            file.write(str(a))
            file.write("\n")
            i = i+1
    file.close()
    tkLabel = tk.Label(tab3, text = "Values have been measured").grid(row = 2, column = 0)
    predict = tk.Button(tab3, text = "Predict", command = predictFun).grid(row = 3, column = 1)
    
win = tk.Tk()                                #Create instance
win.title("Temperature Prediction")          #Set title
win.minsize(350,200)
tabControl = ttk.Notebook(win)               #Create a Tab Control
tab1 = tk.Frame(tabControl)                  #Create a tab
tab2 = tk.Frame(tabControl)                  #Create a tab
tab3 = tk.Frame(tabControl)
tabControl.add(tab1, text = "Register")      #Add the tab
tabControl.add(tab2, text = "Login")         #Add the tab
tabControl.pack(expand = 1, fill = "both")   #Pack to make visible
var=tk.StringVar()

tkLabel = tk.Label(tab1, text = "First Name:").grid(row = 0, column = 0)
fname = tk.Entry(tab1, width = 12)
fname.grid(row = 0, column = 1)   

tkLabel = tk.Label(tab1, text = "Last Name:").grid(row = 1, column = 0)   
lname = tk.Entry(tab1, width = 12)
lname.grid(row = 1, column = 1)   

tkLabel = tk.Label(tab1, text = "Factory Name:").grid(row = 2, column = 0)   
cname = tk.Entry(tab1, width = 12)
cname.grid(row = 2, column = 1)   

tkLabel = tk.Label(tab1, text = "Factory Type:").grid(row = 3, column = 0) 
factType1 = tk.Radiobutton (tab1, text = "Nuclear Plant    ", variable = var, value = 1).grid(row = 3, column = 1) 
factType2 = tk.Radiobutton (tab1, text = "Textile Industry ", variable = var, value = 2).grid(row = 4, column = 1) 
factType3 = tk.Radiobutton (tab1, text = "Thermal Plant    ", variable = var, value = 3).grid(row = 5, column = 1) 
factType4 = tk.Radiobutton (tab1, text = "Chemical Industry", variable = var, value = 4).grid(row = 6, column = 1) 

tkLabel = tk.Label(tab1, text = "Threshold Temperature:").grid(row = 7, column = 0)   
temp = tk.Entry(tab1, width = 12)
temp.grid(row = 7, column = 1) 

tkLabel = tk.Label(tab1, text = "Username:").grid(row = 8, column = 0)   
uname = tk.Entry(tab1, width = 12)
uname.grid(row = 8, column = 1)   

tkLabel = tk.Label(tab1, text = "Password:").grid(row = 9, column = 0)   
password = tk.Entry(tab1, width = 12)
password.grid(row = 9, column = 1)   

register = tk.Button(tab1, text = "Register", command = reg).grid(row = 10, column = 1)

tkLabel = tk.Label(tab2, text = "LOGIN").grid(row = 0, column = 1) 

tkLabel = tk.Label(tab2, text = "Username:").grid(row = 1, column = 0)   
unameLogin = tk.Entry(tab2, width = 12)
unameLogin.grid(row = 1, column = 3)   

tkLabel = tk.Label(tab2, text = "Password:").grid(row = 2, column = 0)   
passwordLogin = tk.Entry(tab2, width = 12)
passwordLogin.grid(row = 2, column = 3) 

login = tk.Button(tab2, text = "Login", command = login).grid(row = 3, column = 2)

measure = tk.Button(tab3, text = "Measure", command = measure).grid(row = 1, column = 1)

#tkLabel.pack(fill=tk.X)
win.mainloop()

In [ ]:
conn = sqlite3.connect('pythonProject.db')
rows = conn.execute('''SELECT * FROM REGISTRATION''')
for i in rows:
    print(i)
conn.close()

In [ ]:
conn = sqlite3.connect('pythonProject.db')
conn.execute('''DELETE FROM REGISTRATION''')

conn.commit()

In [ ]:
file = open("project.csv", "r")
data = file.read()
print(data)